In [25]:
import pandas as pd
import numpy as np
import re
import os
import string


In [26]:
def read_file(directory = "CSE508_Winter2023_Dataset_Processed"):
    data = {}
    for filename in os.listdir(directory):    
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as f:
            file_contents = f.read()
        data[filename] = file_contents
    return data

data = read_file()
print("File count = ",len(data))

keys = list(data.keys())

s = "";
df = []
for key in data:
    df.append(data[key])
    


File count =  1400


In [27]:
from collections import defaultdict

def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    
    
    for doc_id, document in enumerate(documents):
        string1 = document.split()
        for word in range(len(string1)-1):
            bi_string = string1[word:word+2]
            bi = " ".join(bi_string)
            inverted_index[bi].append(doc_id)
    
    return inverted_index

In [28]:
import pickle
from collections import defaultdict

inverted_index = create_inverted_index(df)

def save_inverted_index(inverted_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(inverted_index, f)
        
def load_inverted_index(file_path):
    with open(file_path, 'rb') as f:
        inverted_index = pickle.load(f)
        
    return inverted_index

In [29]:
from collections import defaultdict


pos_index = defaultdict(list)
for no_1, doc in enumerate(df):
    conv_lower = doc.lower()
    t = conv_lower.split()
    for no_2, token_no in enumerate(t):
        if token_no not in pos_index:
            pos_index[token_no] = defaultdict(list)
        if no_2 not in pos_index[token_no]:
            pos_index[token_no][no_1] = []
        pos_index[token_no][no_1].append(no_2)
        
# print(pos_index)

In [30]:
import pickle 
def save_positional_index(pos_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(pos_index, f)
        
def load_positional_index(file_path):
    with open(file_path, 'rb') as f:
        pos_index = pickle.load(f)
    return pos_index